In [1]:
import numpy as np
import random

In [2]:
keys = ['q', 'w', 'e', 'r', 't', 'y', 'u', 'i', 'o', 'p', 'a', 's', 'd', 'f', 'g', 'h', 'j', 'k', 'l', 'z', 'x', 'c', 'v', 'b', 'n', 'm', ';', ',', '.', '/']

In [5]:
def rand_kboard():
    #returns a random keyboard
    keys = ['q', 'w', 'e', 'r', 't', 'y', 'u', 'i', 'o', 'p', 'a', 's', 'd', 'f', 'g', 'h', 'j', 'k', 'l', 'z', 'x', 'c', 'v', 'b', 'n', 'm', ';', ',', '.', '/']
    values = ['q', 'w', 'e', 'r', 't', 'y', 'u', 'i', 'o', 'p', 'a', 's', 'd', 'f', 'g', 'h', 'j', 'k', 'l', 'z', 'x', 'c', 'v', 'b', 'n', 'm', ';', ',', '.', '/']
    kboard = {}

    import random
    for k in range(30):
        n = random.randint(0, len(values)-1)
        kboard[keys[k]] = values[n]
        values.remove(values[n])
    
    return kboard

In [6]:
def full_list():
    #returns a list of all pairs of keys
    full = []
    
    for k1 in keys:
        for k2 in keys:
            if keys.index(k1) < keys.index(k2):
                full.append([k1, k2])
    return full

In [7]:
def is_possible(s, kboard, bf, rm):
    #tells whether swap s results in a keyboard with a possible swap of improvement
    l = swap([s], kboard, bf, rm)
    for pair in full_list():
        m = swap([pair], kboard, bf, rm)
        if m[1] < l[1]:
            return True
    return False

In [11]:
def opti(kboard, x_len, y, bf, rm):
    #main body of the simple hill climbing optimization
    
    #kboard should begin as the output of rand_keyboard()
    #x_len begins at 0 and is the number of steps taken
    #y begins as [] and is the list of scores at each step
    #bf is the dictionary of bigram frequencies
    #rm is the dictionary of key-pair costs
    import matplotlib.pyplot as plt
    
    
    lateral = []
    options = full_list()
    lats = []
    c=0
    while len(options) > 0:
        
        l = swap(options, kboard, bf, rm)
        
        kboard = l[0]
        y.append(l[1])
        
        lateral.append(l[2]['lateral'])
        if list(l[2].values()) == [[], []]:
            pass
        elif l[2]['lateral'] == []:
            try:
                options.remove(l[2]['worse'])
            except:
                pair = l[2]['worse']
                reverse = []
                reverse.append(pair[1])
                reverse.append(pair[0])
                options.remove(reverse)
        elif l[2]['worse'] == []:
            options.remove(l[2]['lateral'])
            lats.append(l[2]['lateral'])            
        
        x_len += 1
        c += 1
        
    possible = []
    for s in lats:
        if is_possible(s, kboard, bf, rm):
            possible.append(s)
        
    if len(possible) == 0:
        x = range(x_len)
        for lm in x:
            lm += 1
        
        plt.scatter(x, y, alpha=0.9)
        plt.show()
        print(y[-1])
    
        return kboard
    
    else:
        pair = random.choice(possible)
        pair = [pair[0] + pair[1]]
        kboard = swap(pair, kboard, bf, rm)[0]
        return opti(kboard, x_len, y, bf, rm)

In [9]:
def swap(options, kboard, bf, rm):
    #returns list including a keyboard with a pair of keys swapped given in the list options, its score, and if that swap is bad or neutral
    
    import random
    
    s1 = score(kboard, bf, rm)
    s_kboard = {}
    for k in kboard:
        s_kboard[k] = kboard[k]
    
    i = random.randint(0, len(options)-1)
    
    k1 = options[i][0]
    k2 = options[i][1]

    s_kboard[k1] = kboard[k2]
    s_kboard[k2] = kboard[k1]

    
    s2 = score(s_kboard, bf, rm)

    if s2 < s1:
        return [s_kboard, s2, {'lateral':[], 'worse':[]}]
    
    if s2 > s1:
        return [kboard, s1, {'lateral':[], 'worse':[k1, k2]}]
    
    if s2 == s1:
        return [kboard, s1, {'lateral':[k1, k2], 'worse':[]}]

In [10]:
def score(kboard, bigram_freqs, key_times):
    #returns the score of a keyboard
    nogo = [';', ',', '.', '/']
    t = 0
    for i in keys:
        char1 = kboard[i]
        for j in keys:
            char2 = kboard[j]
            if char1 not in nogo and char2 not in nogo:
                freq = bigram_freqs[char1 + char2]
            else:
                freq = 0
            try:
                cost = key_times[i + j]
            except:
                cost = key_times[j + i]
            
            t += float(cost) * freq
    return t